### 1. Importar Librerías:

In [42]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

### 2. Abrir Archivo:

In [ ]:
# dataset descargado de https://climate.cityofnewyork.us/initiatives/nyc-greenhouse-gas-inventories/
df = pd.read_excel('../Datasets\CY_2005_CY_2022_citywide.xlsx')

SWCV (Small Waste Collection Vehicles - Vehículos de Pequeña Recolección de Residuos)

Ahora vamos a seleccionar las columnas de interés: las emisiones en toneladas de CO2 para cada año disponible.

Aclaración: <br>
- 'CY XXXX Consumed' corresponde a las "VMT" ("Vehicle Miles Traveled" = "Millas Recorridas por Vehículo"). Es una medida utilizada para cuantificar la cantidad total de millas que se han recorrido por vehículos en un área determinada (NYC) durante un período de tiempo específico (Año XXXX). <br>
- 'CY XXXX tCO2e' corresponde a las toneladas de CO2 emitidas en NYC durante el año XXXX. <br>
- 'CY XXXX Source MMBtu' Esta columna se refiere a los millones de unidades térmicas británicas (MMBtu) consumidas por la fuente en el año XXXX. El MMBtu es una medida de energía y es comúnmente utilizada para medir la cantidad de energía que se consume.
- CY XXXX tCO2e 20 yr GWP: Esta columna indica las emisiones de CO2 equivalente (tCO2e) para el año XXXX, calculadas utilizando un Potencial de Calentamiento Global (GWP, por sus siglas en inglés) de 20 años. El GWP es una medida que compara el potencial de calentamiento global de un gas en particular con el dióxido de carbono (CO2) durante un período de tiempo específico (en este caso, 20 años).


### 3. Normalización de tabla (desnaturalización):

In [ ]:
# Seleccionamos las columnas que vamos a utilizar
columns_to_keep = ['Sectors Sector', 'Category Full', 'Category Label'] + [col for col in df.columns if col.startswith('CY')]

## Seleccionar las columnas que vamos a utilizar
columns_to_keep = ['Sectors Sector', 'Category Full', 'Category Label'] + [col for col in df.columns if col.startswith('CY')]

# Crear un nuevo DataFrame con las columnas seleccionadas
df_normalized = df[columns_to_keep].copy()

# Realizar la operación de fundido (melt)
df_normalized = pd.melt(df_normalized, id_vars=['Sectors Sector', 'Category Full', 'Category Label'], var_name='Año', value_name='Valor')

####

# Seleccionamos las columnas que vamos a utilizar
columns_to_keep = ['Sectors Sector', 'Category Full', 'Category Label'] + [col for col in df.columns if col.startswith('CY')]

# Crear un nuevo DataFrame con las columnas seleccionadas
df_normalized = df[columns_to_keep].copy()

# Realizar la operación de fundido (melt)
df_normalized = pd.melt(df_normalized, id_vars=['Sectors Sector', 'Category Full', 'Category Label'], var_name='Año', value_name='Valor')

# Crear una columna auxiliar 'Tipo' y procesar la columna 'Año'
df_normalized['Tipo'] = df_normalized['Año'].apply(lambda x: re.findall(r'(?:\d{4})\s(.+)', x)[0])
df_normalized['Año'] = df_normalized['Año'].apply(lambda x: re.findall(r'(\d{4})', x)[0])


### 4. Normalizar columna Tipo:

In [ ]:
# Filtrar solo las filas donde el tipo es 'Consumed' o 'tCO2e'
df_normalized = df_normalized[df_normalized['Tipo'].isin(['Consumed', 'tCO2e','Source MMBtu'])]

# Utilizar pivot_table para convertir las categorías de 'Tipo' en columnas
df_pivot = df_normalized.pivot_table(index=['Sectors Sector', 'Category Full', 'Category Label', 'Año'],
                                     columns='Tipo', values='Valor', aggfunc='sum').reset_index()


### 5. Eliminar Nulos y duplicados (de existir):

In [ ]:
# Eliminar registros nulos
df_pivot.dropna(inplace=True)

# Eliminar registros duplicados
df_pivot.drop_duplicates(inplace=True)

In [56]:
#df_pivot.to_csv('../DataClean/CO2_NYC.csv')